## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the `raw_sales` table from the database `retail_sales` fon of Ironhack's databases. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

### Exploring and cleaning data

In [1]:
# import required libraries
import pandas as pd
import numpy as np

In [2]:
# import database
raw_sales = pd.read_csv("../Datasets as CSV/retail_sales-raw_sales.csv", sep=";")
raw_sales_original = raw_sales.copy()
raw_sales

,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04 00:00:00,29,1469,1199.0,1.0
1,2015-01-04 00:00:00,28,21364,479.0,1.0
2,2015-01-04 00:00:00,28,21365,999.0,2.0
3,2015-01-04 00:00:00,28,22104,249.0,2.0
4,2015-01-04 00:00:00,28,22091,179.0,1.0
...,...,...,...,...,...
4540,2015-01-04 00:00:00,15,4240,1299.0,1.0
4541,2015-01-04 00:00:00,14,21922,99.0,1.0
4542,2015-01-04 00:00:00,15,1969,3999.0,1.0
4543,2015-01-04 00:00:00,14,22091,179.0,1.0


In [3]:
raw_sales.info()
# No NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          4545 non-null   object 
 1   shop_id       4545 non-null   int64  
 2   item_id       4545 non-null   int64  
 3   item_price    4545 non-null   float64
 4   item_cnt_day  4545 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 177.7+ KB


In [4]:
raw_sales.describe()

,shop_id,item_id,item_price,item_cnt_day
count,4545.000000,4545.000000,4545.000000,4545.000000
mean,34.021122,11140.459406,1031.686121,1.103630
std,16.565517,6558.649572,2073.919990,0.536967
min,2.000000,30.000000,3.000000,-1.000000
25%,22.000000,4977.000000,249.000000,1.000000
50%,31.000000,11247.000000,479.000000,1.000000
75%,50.000000,16671.000000,1192.000000,1.000000
max,59.000000,22162.000000,27990.000000,10.000000


### Creating aggregates

In [17]:
# Adding column with total price by item by store
raw_sales["total_price"]=raw_sales["item_price"]*raw_sales["item_cnt_day"]

# Checking for entries where item_cnt_day = 0, to remove them --> there are not
raw_sales[raw_sales["item_cnt_day"]==0]


,date,shop_id,item_id,item_price,item_cnt_day,total_price


In [18]:
raw_sales.head()

,date,shop_id,item_id,item_price,item_cnt_day,total_price
0,2015-01-04 00:00:00,29,1469,1199.0,1.0,1199.0
1,2015-01-04 00:00:00,28,21364,479.0,1.0,479.0
2,2015-01-04 00:00:00,28,21365,999.0,2.0,1998.0
3,2015-01-04 00:00:00,28,22104,249.0,2.0,498.0
4,2015-01-04 00:00:00,28,22091,179.0,1.0,179.0


#### Aggregate per store

In [12]:
# New aggregate w/ total sales per store per day
agg_store = raw_sales.groupby("shop_id").total_price.sum().to_frame()
agg_store.to_csv("../Datasets as CSV/agg_store.csv", sep=";")
agg_store.head()

,total_price
shop_id,
2,103746.0
3,67443.0
4,29361.0
5,33138.0
6,138678.0


#### Aggregate per item

In [14]:
# New aggregate w/ total sales per item per day
agg_item = raw_sales.groupby("item_id").total_price.sum().to_frame()
agg_store.to_csv("../Datasets as CSV/agg_item.csv", sep=";")
agg_item.head()

,total_price
item_id,
30,507.0
31,1089.0
32,447.0
42,897.0
59,747.0
